In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../../main_data.csv")

In [ ]:
df.sample(5)

,id_checkin,nome_aluno,idade_aluno,data_checkin,data_checkout,data_inicio,data_fim,valor_plano,nome_plano,tipo_plano,churn
145740,151965,Dorolisa Canland,26,2025-01-05 10:09:47,2025-01-05 11:03:47,2024-08-20 00:00:00,2025-12-06 00:00:00,69.9,Beginner,Mensal,0
526425,548777,Winnie O'Cleary,57,2025-01-16 12:24:16,2025-01-16 13:11:16,2025-01-01 00:00:00,NaN,159.9,Focus,Semestral,1
62132,64572,Billi Folbigg,55,2024-12-13 10:11:55,2024-12-13 11:05:55,2024-02-09 00:00:00,NaN,69.9,Beginner,Mensal,1
34305,35559,Ardine Stivey,44,2024-11-21 01:20:14,2024-11-21 02:04:14,2024-09-06 00:00:00,2025-10-06 00:00:00,69.9,Beginner,Mensal,0
298885,311836,Korey Freyn,25,2024-10-01 19:55:58,2024-10-01 20:45:58,2024-01-25 00:00:00,NaN,159.9,Focus,Semestral,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535299 entries, 0 to 535298
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_checkin     535299 non-null  int64  
 1   nome_aluno     535299 non-null  object 
 2   idade_aluno    535299 non-null  int64  
 3   data_checkin   535299 non-null  object 
 4   data_checkout  535299 non-null  object 
 5   data_inicio    535299 non-null  object 
 6   data_fim       143435 non-null  object 
 7   valor_plano    535299 non-null  float64
 8   nome_plano     535299 non-null  object 
 9   tipo_plano     535299 non-null  object 
 10  churn          535299 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 44.9+ MB


### Conhecendo as colunas - coluna por coluna

Esses dados são fictícios, portanto, algumas coisas podem não fazer sentido para o modelo, mas eu procurei seguir uma lógica e uma certa linearidade para fazer com que o meu modelo aprenda os padrões.


In [ ]:
len(df["nome_aluno"].unique()) # todos os nomes são unicos

6575

### Sobre o range das idades:
Eu gerei a idade com mínimo de 16 anos e máximo de 65 anos.

*Obs: não frequento acadêmia atualmente, mas eu imagino que seja nesse range ainda rs*

In [ ]:
df["idade_aluno"].describe()

,idade_aluno
count,535299.000000
mean,40.027136
std,14.099250
min,16.000000
25%,28.000000
50%,40.000000
75%,52.000000
max,64.000000


Sobre as **datas de checkin** (foram pedidas neste teste) e as **datas de checkout**, que não foram pedidas mas achei necessário pois, o modelo deverá aprender com a feature de duração média das visitas.

In [ ]:
print("dtype data_checkin:", df["data_checkin"].dtype)
print("dtype data_checkout:", df["data_checkout"].dtype)

# depois vamos converter para os datetimes corretos

dtype data_checkin: object
dtype data_checkout: object


### Outras datas
As datas de inicio e fim não foram pedidas mas eu inclui mesmo assim, pois achei importante. Usei uma lógica para identificar churn por parte dos alunos que não tem matrícula a partir de hoje (03/06/2025), então, se o aluno tem data_fim, antes disso, é churn, caso contrário, tem matricula ativa.

In [ ]:
# a distribuição do target ficou assim então:
df["churn"].value_counts(normalize=True)



# para dados fictícios, até que ficou parecido com casos reais rs

,proportion
churn,
1,0.732047
0,0.267953


### Os planos

A forma como escolhi trabalhar com os planos foi detalhada dessa forma:

**Nomes dos planos**

- Beginner
- Focus

obs: tirei da minha imaginação

**Valores**

- Beginner: 69.90
- Focus: 159.90


**Tipo de plano**

Trabalhei com tipos mensais e semestrais(estou trabalhando com dados com range de 1 ano). Fui bem direto quanto a isso. Qualquer Beginner é mensal, qualquer Focus é semestral. Sei que isso não reflete a vida real e poderia enviezar o modelo se ele tivesse que ser treinado com as features dos valores.

In [ ]:
df[["valor_plano", "nome_plano", "tipo_plano"]].sample(10)

,valor_plano,nome_plano,tipo_plano
123149,159.9,Focus,Semestral
323851,159.9,Focus,Semestral
298055,69.9,Beginner,Mensal
90690,159.9,Focus,Semestral
3160,69.9,Beginner,Mensal
228863,69.9,Beginner,Mensal
85021,159.9,Focus,Semestral
467672,69.9,Beginner,Mensal
261462,69.9,Beginner,Mensal
131158,159.9,Focus,Semestral


Aqui vamos converter para datetime essas colunas de datas para preparar os dados para modelagem.

In [ ]:
cols_to_convert = ["data_checkin", "data_checkout", "data_inicio", "data_fim"]
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_datetime)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535299 entries, 0 to 535298
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id_checkin     535299 non-null  int64         
 1   nome_aluno     535299 non-null  object        
 2   idade_aluno    535299 non-null  int64         
 3   data_checkin   535299 non-null  datetime64[ns]
 4   data_checkout  535299 non-null  datetime64[ns]
 5   data_inicio    535299 non-null  datetime64[ns]
 6   data_fim       143435 non-null  datetime64[ns]
 7   valor_plano    535299 non-null  float64       
 8   nome_plano     535299 non-null  object        
 9   tipo_plano     535299 non-null  object        
 10  churn          535299 non-null  int64         
dtypes: datetime64[ns](4), float64(1), int64(3), object(3)
memory usage: 44.9+ MB


In [ ]:
df.sample(5)

,id_checkin,nome_aluno,idade_aluno,data_checkin,data_checkout,data_inicio,data_fim,valor_plano,nome_plano,tipo_plano,churn
338465,352997,Marcelle Chessum,24,2025-03-26 21:08:50,2025-03-26 21:55:50,2024-10-10,NaT,69.9,Beginner,Mensal,1
151879,158702,Ed Robecon,58,2025-03-26 14:01:51,2025-03-26 14:45:51,2025-03-24,NaT,159.9,Focus,Semestral,1
225157,234949,Hart Baber,37,2025-02-22 22:00:29,2025-02-22 22:58:29,2025-01-28,NaT,69.9,Beginner,Mensal,1
155887,162812,Eleonora Merricks,35,2024-11-01 16:28:37,2024-11-01 17:28:37,2024-10-09,NaT,69.9,Beginner,Mensal,1
318617,332227,Lilias Karolowski,21,2024-11-17 13:24:54,2024-11-17 14:04:54,2024-08-28,NaT,159.9,Focus,Semestral,1


In [ ]:
import numpy as np

In [ ]:
# 1. Frequência semanal
df["data_checkin"] = pd.to_datetime(df["data_checkin"])
df["data_inicio"] = pd.to_datetime(df["data_inicio"])

In [ ]:
ultimo_checkin = df.groupby("nome_aluno")["data_checkin"].max().reset_index()
ultimo_checkin.rename(columns={"data_checkin": "ultimo_checkin"}, inplace=True)

In [ ]:
df = df.merge(ultimo_checkin, on="nome_aluno", how="left")

In [ ]:
df["semanas_matriculado"] = (
    (df["ultimo_checkin_x"] - df["data_inicio"]).dt.days / 7
).clip(lower=1)

In [ ]:
checkins_por_aluno = df.groupby("nome_aluno")["id_checkin"].count().reset_index()
checkins_por_aluno.rename(columns={"id_checkin": "total_checkins"}, inplace=True)

In [ ]:
df = df.merge(checkins_por_aluno, on="nome_aluno", how="left")

In [ ]:
df["frequencia_semanal"] = df["total_checkins"] / df["semanas_matriculado"]

In [ ]:
df["frequencia_semanal"].sample(5)

,frequencia_semanal
525893,3.125461
512222,20.475000
316177,2.200758
324388,0.370861
29619,2.220339


In [ ]:
# 2. Tempo desde o último checkin

df["ultimo_checkin"] = df.groupby("nome_aluno")["data_checkin"].transform("max")
df["tempo_desde_ultimo_checkin"] = (df["ultimo_checkin"] - df["data_checkin"]).dt.days

In [ ]:
# 3. Duração média das visitas

df["duracao_visita"] = (df["data_checkout"] - df["data_checkin"]).dt.total_seconds() / 60  # Minutos
duracao_media = df.groupby("nome_aluno")["duracao_visita"].mean().reset_index()
duracao_media.rename(columns={"duracao_visita": "duracao_media_visitas"}, inplace=True)

df = df.merge(duracao_media, on="nome_aluno", how="left")

In [ ]:
# 4. Tipo de plano
df["tipo_plano"].value_counts(normalize=True)

,proportion
tipo_plano,
Mensal,0.561892
Semestral,0.438108


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535299 entries, 0 to 535298
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   id_checkin                  535299 non-null  int64         
 1   nome_aluno                  535299 non-null  object        
 2   idade_aluno                 535299 non-null  int64         
 3   data_checkin                535299 non-null  datetime64[ns]
 4   data_checkout               535299 non-null  datetime64[ns]
 5   data_inicio                 535299 non-null  datetime64[ns]
 6   data_fim                    143435 non-null  datetime64[ns]
 7   valor_plano                 535299 non-null  float64       
 8   nome_plano                  535299 non-null  object        
 9   tipo_plano                  535299 non-null  object        
 10  churn                       535299 non-null  int64         
 11  semana_checkin              535299 non-

In [ ]:
df_train = df[[
    "frequencia_semanal",
    "tempo_desde_ultimo_checkin",
    "duracao_media_visitas",
    "tipo_plano",
    "churn"
]]

In [ ]:
df_train["frequencia_semanal"].value_counts()

,count
frequencia_semanal,
7.000000,1270
3.500000,1146
1.750000,815
2.333333,642
4.000000,640
...,...
0.373333,4
0.682927,4
0.600000,3


In [ ]:
df_train["tempo_desde_ultimo_checkin"].describe()

,tempo_desde_ultimo_checkin
count,535299.000000
mean,106.958117
std,97.564007
min,0.000000
25%,27.000000
50%,78.000000
75%,163.000000
max,454.000000


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535299 entries, 0 to 535298
Data columns (total 5 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   frequencia_semanal          535299 non-null  float64
 1   tempo_desde_ultimo_checkin  535299 non-null  int64  
 2   duracao_media_visitas       535299 non-null  float64
 3   tipo_plano                  535299 non-null  object 
 4   churn                       535299 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 20.4+ MB


Agora vamos usar One-hot encoding na coluna categorica `tipo_plano`. Na minha opnião, faz mais sentido e é mais simples, nesse caso, trabalhar com get_dummies do pandas mesmo.

In [ ]:
df_train = pd.get_dummies(df_train, columns=['tipo_plano'], dtype=int)

In [ ]:
df_train = df_train[[
    "frequencia_semanal",
    "tempo_desde_ultimo_checkin",
    "duracao_media_visitas",
    "tipo_plano_Mensal",
    "tipo_plano_Semestral",
    "churn"
]]

In [ ]:
df_train.sample(5)

,frequencia_semanal,tempo_desde_ultimo_checkin,duracao_media_visitas,tipo_plano_Mensal,tipo_plano_Semestral,churn
78254,6.611111,0,49.470588,0,1,1
160301,6.670588,41,49.987654,1,0,0
65431,4.555556,123,50.000000,1,0,1
137211,2.234940,148,49.924528,1,0,1
489251,2.809211,152,50.114754,0,1,1


In [ ]:
df_train["frequencia_semanal"].value_counts()

,count
frequencia_semanal,
7.000000,1270
3.500000,1146
1.750000,815
2.333333,642
4.000000,640
...,...
0.373333,4
0.682927,4
0.600000,3


In [ ]:
labels_categorias = [
    "nivel1",
    "nivel2",
    "nivel3",
    "nivel4"
]

In [ ]:
df_train['frequencia_semanal_cat'] = pd.qcut(
  df_train['frequencia_semanal'],
  q=4,
  labels=labels_categorias,
  duplicates='drop'
)

In [ ]:
df_train['tempo_desde_ultimo_checkin_cat'] = pd.qcut(
  df_train['tempo_desde_ultimo_checkin'],
  q=4,
  labels=labels_categorias,
  duplicates='drop'
)

In [ ]:
df_train['duracao_media_visitas_cat'] = pd.qcut(
  df_train['duracao_media_visitas'],
  q=4,
  labels=labels_categorias,
  duplicates='drop'
)

In [ ]:
df_train.sample(5)

,frequencia_semanal,tempo_desde_ultimo_checkin,duracao_media_visitas,tipo_plano_Mensal,tipo_plano_Semestral,churn,frequencia_semanal_cat,tempo_desde_ultimo_checkin_cat,duracao_media_visitas_cat
253758,1.513953,25,50.752688,0,1,1,nivel1,nivel1,nivel4
527863,3.142336,41,50.097561,1,0,0,nivel2,nivel2,nivel3
191656,0.813953,215,50.480000,1,0,0,nivel1,nivel4,nivel4
474470,35.636364,15,50.526786,1,0,1,nivel4,nivel1,nivel4
268604,1.947826,175,50.166667,0,1,1,nivel1,nivel4,nivel3


In [ ]:
colunas_para_codificar = [
    'frequencia_semanal_cat',
    'tempo_desde_ultimo_checkin_cat',
    'duracao_media_visitas_cat'
]

In [ ]:
mapeamento_ordinal = {
    "nivel1": 0,
    "nivel2": 1,
    "nivel3": 2,
    "nivel4": 3
}

In [ ]:
for col in colunas_para_codificar:
    nova_coluna_nome = col.replace('_cat', '_encoded_map') # nome diferente para comparação

    df_train[nova_coluna_nome] = df_train[col].map(mapeamento_ordinal)

In [ ]:
df_train.sample(5)

,frequencia_semanal,tempo_desde_ultimo_checkin,duracao_media_visitas,tipo_plano_Mensal,tipo_plano_Semestral,churn,frequencia_semanal_cat,tempo_desde_ultimo_checkin_cat,duracao_media_visitas_cat,frequencia_semanal_encoded_map,tempo_desde_ultimo_checkin_encoded_map,duracao_media_visitas_encoded_map
15086,2.145161,271,49.982456,1,0,1,nivel2,nivel4,nivel2,1,3,1
432994,3.500000,98,50.000000,1,0,1,nivel3,nivel3,nivel2,2,2,1
429517,4.743169,34,50.137097,1,0,1,nivel3,nivel2,nivel3,2,1,2
23039,2.408602,264,49.976562,1,0,0,nivel2,nivel4,nivel2,1,3,1
206912,4.666667,199,50.850000,1,0,0,nivel3,nivel4,nivel4,2,3,3


In [ ]:
df_train_cat = df_train[[
    "frequencia_semanal_encoded_map",
    "tempo_desde_ultimo_checkin_encoded_map",
    "duracao_media_visitas_encoded_map",
    "tipo_plano_Mensal",
    "tipo_plano_Semestral",
    "churn"
]]

In [ ]:
df_train_num = df_train[[
    "frequencia_semanal",
    "tempo_desde_ultimo_checkin",
    "duracao_media_visitas",
    "tipo_plano_Mensal",
    "tipo_plano_Semestral",
    "churn"
]]

In [ ]:
df_train.to_csv("../../data/general.csv", index=False)
df_train_cat.to_csv("../../data/cat.csv", index=False)
df_train_num.to_csv("../../data/num.csv", index=False)